In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import datetime
from zoneinfo import ZoneInfo

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (41 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}

In [5]:
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm

try:
    root_agent = Agent(
        name="weather_time_agent",
        model=LiteLlm("gemini/gemini-2.0-flash"),
        description=(
            "Agent to answer questions about the time and weather in a city."
        ),
        instruction=(
            "You are a helpful agent who can answer user questions about the time and weather in a city."
        ),
        tools=[get_weather, get_current_time],
    )
except Exception as e:
    print(f"Counld not create Agent. Error: {e}")

In [6]:
from google.adk.sessions import InMemorySessionService

session_service = InMemorySessionService()

session = session_service.create_session(
    app_name="weather_time_agent",
    user_id="user_1",
    session_id="session_1"
)

In [7]:
from google.adk.runners import Runner

runner = Runner(
    agent=root_agent,
    app_name="weather_time_agent",
    session_service=session_service,
)

In [8]:
import asyncio
from google.genai import types

async def call_agent_async(query: str):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")

    # Prepare the user's message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response." # Default

    # Key Concept: run_async executes the agent logic and yields Events.
    # We iterate through events to find the final answer.
    async for event in runner.run_async(user_id="user_1", session_id="session_1", new_message=content):
        # You can uncomment the line below to see *all* events during execution
        # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            # Add more checks here if needed (e.g., specific error codes)
            break # Stop processing events once the final response is found

    print(f"<<< Agent Response: {final_response_text}")

In [9]:
async def run_conversation():
    await call_agent_async("What is the weather like in London?")
    await call_agent_async("How about Paris?") # Expecting the tool's error message
    await call_agent_async("Tell me the weather in New York")
    await call_agent_async("What time is it in New York?")

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: What is the weather like in London?
<<< Agent Response: I am sorry, I cannot get the weather information for London. There was an error.

>>> User Query: How about Paris?
<<< Agent Response: I am sorry, I cannot get the weather information for Paris. There was an error.

>>> User Query: Tell me the weather in New York
<<< Agent Response: OK. The weather in New York is sunny with a temperature of 25 degrees Celsius (41 degrees Fahrenheit).


>>> User Query: What time is it in New York?
<<< Agent Response: The current time in New York is 2025-04-20 11:30:19 EDT-0400.
